## Recolectando y entendiendo los datos

In [1]:
#dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
import pandas as pd
#Leer los datos
Email_Data = pd.read_csv("spam.csv",encoding ='latin1')
#Revisar los datos
Email_Data.columns

Email_Data = Email_Data[['v1', 'v2']]
Email_Data = Email_Data.rename(columns={"v1":"Target", "v2":"Email"})
Email_Data.head()
#ham = jamon = bueno , Spam =condimentado =  malo

,Target,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


##Preproceso del texto y generación de una representación (feature engineering)

In [2]:
#import
import numpy as np
import matplotlib.pyplot as plt
import string


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#se importan las funciones de algunas bibliotecas
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from nltk.stem import PorterStemmer
from textblob import TextBlob # ver mas en:https://textblob.readthedocs.io/en/dev/quickstart.html
from textblob import Word

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [5]:
#preproceso
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: " ".join(x.lower() for x in x.split()))
stop = stopwords.words('english')
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
st = PorterStemmer()
# experimente con stemming y lematizacion o con ambos. ¿cual da mejores resultados?
Email_Data['Email'] = Email_Data['Email'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#Email_Data['Email'] = Email_Data['Email'].apply(lambda x: " ".join([Word (word).lemmatize() for word in x.split()]))
Email_Data.head()


,Target,Email
0,ham,"go jurong point, crazy.. avail bugi n great wo..."
1,ham,ok lar... joke wif u oni...
2,spam,free entri 2 wkli comp win fa cup final tkt 21...
3,ham,u dun say earli hor... u c alreadi say...
4,ham,"nah think goe usf, live around though"


In [6]:
# Se divide el texto en test y trainning y validatión
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(Email_Data['Email'], Email_Data['Target'])

# TFIDF Generacion de caracteristicas para un máximo de 5000 caracteristicas.
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',
max_features=5000)
#tfidf_vect.fit(Email_Data['Email']) # Error no se debe hacer sobre todo el dataset. se filtra informacion del test al modelo
tfidf_vect.fit(train_x) # se debe hacer sobre el train
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)
xtrain_tfidf.data

array([0.18662522, 0.1941594 , 0.26050101, ..., 0.25221756, 0.24322491,
       0.22713869])

##Entrenamiento

In [7]:
# Esta función general permite entrenar cualquier modelo. Es un modelo base usando parametros de los modelos preestablecidos
def train_model(classifier, feature_vector_train, label,feature_vector_valid, is_neural_net=False):
# Se entrena el clasificador
  classifier.fit(feature_vector_train, label)
  # hace predicción sobre el conjunto valid.
  predictions = classifier.predict(feature_vector_valid)
  return metrics.accuracy_score(predictions, valid_y)

In [8]:
# Modelo Naive Bayes
accuracy = train_model(naive_bayes.MultinomialNB(alpha=0.2), xtrain_tfidf,
train_y, xvalid_tfidf)
print ("Accuracy: ", accuracy)

Accuracy:  0.9856424982053122


In [9]:
# Modelo de regresión logistica
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf,
train_y, xvalid_tfidf)
print ("Accuracy: ", accuracy)

Accuracy:  0.9698492462311558


In [10]:
# Modelo Support Vector Machine
accuracy = train_model(svm.SVC(), xtrain_tfidf,
train_y, xvalid_tfidf)
print ("Accuracy: ", accuracy)

Accuracy:  0.9849246231155779
